# Calculate additional performance metrics: Accuracy, balanced accuracy, sensitivity, specificity

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import roc_curve, balanced_accuracy_score, accuracy_score
import sklearn

import src.data.var_names as var_names
from definitions import REPO_ROOT, PROCESSED_DATA_DIR, RAW_DATA_DIR, RESULTS_DIR

In [2]:
abcd_data = pd.read_csv(PROCESSED_DATA_DIR / 'abcd_data.csv')
abcd_data.index = abcd_data['src_subject_id']

In [3]:
diagnoses_to_check = ['Bipolar Disorder', 'ADHD', 'Oppositional Defiant Disorder']

In [4]:
bacc_sens_spec = {
    d: {'accuracy': [], 'balanced_acc': [], 'sensitivity': [], 'specificity': []}
    for d in diagnoses_to_check
}

for i in range(1, 16):
    folder = f"run_unpermuted_seed{1000+i}n2k5"
    filepath = RESULTS_DIR / folder / 'unpermuted' / 'xgboost_cce' / 'test'
    for fold_i in range(10):
        predictions = pd.read_csv(filepath / f"fold_{fold_i}.csv")
        predictions.index = predictions['src_subject_id']
        for diagnosis in diagnoses_to_check:
            y_true, y_pred = abcd_data[diagnosis].align(other=predictions[diagnosis], join='inner')
            fpr, tpr, threshold = roc_curve(
                y_true=y_true,
                y_score=y_pred
            )
            # Compute optimal threshold value, corresponding to maximum
            # Youden index
            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = threshold[optimal_idx]
            # Compute sensitivity and specificity
            sensitivity = tpr[optimal_idx]
            specificity = 1 - fpr[optimal_idx]
            # Apply threshold to predictions and calculate balanced accuracy
            binary_predictions = predictions[diagnosis] >= optimal_threshold
            balanced_accuracy = balanced_accuracy_score(
                y_true=y_true,
                y_pred=binary_predictions
            )
            accuracy = accuracy_score(
                y_true=y_true,
                y_pred=binary_predictions
            )
            # Store values
            bacc_sens_spec[diagnosis]['accuracy'].append(accuracy)
            bacc_sens_spec[diagnosis]['balanced_acc'].append(balanced_accuracy)
            bacc_sens_spec[diagnosis]['sensitivity'].append(sensitivity)
            bacc_sens_spec[diagnosis]['specificity'].append(specificity)

In [5]:
for diagnosis in diagnoses_to_check:
    print(diagnosis)
    print(f"\tAccuracy: "
          f"\t\t{np.mean(bacc_sens_spec[diagnosis]['accuracy']):.4f} "
          f"({min(bacc_sens_spec[diagnosis]['accuracy']):.4f} - "
          f"{max(bacc_sens_spec[diagnosis]['accuracy']):.4f})")
    print(f"\tBalanced accuracy: "
          f"\t{np.mean(bacc_sens_spec[diagnosis]['balanced_acc']):.4f} "
          f"({min(bacc_sens_spec[diagnosis]['balanced_acc']):.4f} - "
          f"{max(bacc_sens_spec[diagnosis]['balanced_acc']):.4f})")
    print(f"\tSensitivity: "
          f"\t\t{np.mean(bacc_sens_spec[diagnosis]['sensitivity']):.4f} "
          f"({min(bacc_sens_spec[diagnosis]['sensitivity']):.4f} - "
          f"{max(bacc_sens_spec[diagnosis]['sensitivity']):.4f})")
    print(f"\tSpecificity: "
          f"\t\t{np.mean(bacc_sens_spec[diagnosis]['specificity']):.4f} "
          f"({min(bacc_sens_spec[diagnosis]['specificity']):.4f} - "
          f"{max(bacc_sens_spec[diagnosis]['specificity']):.4f})")

Bipolar Disorder
	Accuracy: 		0.5978 (0.2473 - 0.8727)
	Balanced accuracy: 	0.5613 (0.5234 - 0.6111)
	Sensitivity: 		0.5190 (0.1386 - 0.9208)
	Specificity: 		0.6037 (0.1942 - 0.9306)
ADHD
	Accuracy: 		0.5557 (0.3297 - 0.7650)
	Balanced accuracy: 	0.5564 (0.5247 - 0.5906)
	Sensitivity: 		0.5577 (0.1615 - 0.8969)
	Specificity: 		0.5551 (0.1971 - 0.9047)
Oppositional Defiant Disorder
	Accuracy: 		0.5498 (0.2169 - 0.8395)
	Balanced accuracy: 	0.5420 (0.5132 - 0.5795)
	Sensitivity: 		0.5308 (0.0485 - 0.9655)
	Specificity: 		0.5532 (0.0884 - 0.9779)
